##This notebook explains the way to utilize SQLAlchemy to upload the structured data into a Snowflake database

##**Terminal Commands:**


**Step 1:** Connect to your snowflake account

- snowsql -a CKMFZWO-AFA59273 -u akshitapathania

**Step 2:** Enter Password

- {password}

**Step 3:** Create a Database called 'TEXT_EXTRACTION'

- CREATE OR REPLACE DATABASE TEXT_EXTRACTION;

(To see your created database along with the schema:

SELECT CURRENT_DATABASE(), CURRENT_SCHEMA();

)

**Step 4:** Create a Warehouse

- CREATE OR REPLACE WAREHOUSE text_extraction_wh WITH
                  WAREHOUSE_SIZE='X-SMALL'
                  AUTO_SUSPEND = 180
                  AUTO_RESUME = TRUE
                  INITIALLY_SUSPENDED=TRUE;

(To see your created warehouse:

SELECT CURRENT_WAREHOUSE();

)

##**Perform step 5 and 6 once the code below is executed on Google Colab:**

**Step 5:** Showcase all the tables present

- SHOW TABLES;

**Step 6:** Display values in the table

- SELECT * FROM STRUCTURED_DATA;


Ensuring that we have access to a Snowflake account and download SQLAlchemy.

In [ ]:
pip install SQLAlchemy snowflake-sqlalchemy snowflake-connector-python

Import the required libraries

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import pandas as pd

Using SQLAlchemy to connect to our Snowflake account by providing connection details such as username, password, account name, database name, schema name, and warehouse name.

In [ ]:
snowflake_username = 'akshitapathania'
snowflake_password = '***********'
snowflake_account = 'CKMFZWO-AFA59273'
snowflake_database = 'text_extraction'
snowflake_schema = 'public'
snowflake_warehouse = 'text_extraction_wh'

# SQLAlchemy Snowflake connection string
connection_string = f'snowflake://{snowflake_username}:{snowflake_password}@{snowflake_account}/{snowflake_database}?warehouse={snowflake_warehouse}&schema={snowflake_schema}'


Preparing the data that we want to upload into Snowflake. This will be structured data that we have extracted from the website and the PDFs.

In [ ]:
# Create SQLAlchemy engine
engine = create_engine(connection_string)


Next, we define the table structure in Snowflake. This involves creating a table with appropriate columns to store data.

In [ ]:
# Define metadata
metadata = MetaData()

# Define the table structure
table_name = 'structured_data'
table = Table(
    table_name,
    metadata,
    Column('id', Integer, primary_key=True),
    Column('topic_name', String),
    Column('year', String),
    Column('level', String),
    Column('introduction_summary', String),
    Column('learning_outcomes', String),
    Column('summary_page_link', String),
    Column('pdf_file_link', String)
)

Once we have defined the table structure, we create the table in Snowflake using SQLAlchemy's create_all() method.

In [ ]:
# Create table in Snowflake if it doesn't exist
if not engine.dialect.has_table(engine, table_name):
    metadata.create_all(engine)

We then insert the data into the Snowflake table created. We used SQLAlchemy's execute() method to execute an INSERT SQL statement.

In [ ]:
# Function to insert data into Snowflake
def insert_data(data):
    with engine.connect() as conn:
        conn.execute(table.insert().values(data))


After inserting the data, commit the changes to persist them in Snowflake.

In [ ]:
# Path to your CSV file
csv_file_path = 'refresher_readings.csv'

# Read data from CSV file
data_from_csv = pd.read_csv(csv_file_path).fillna('')

In [ ]:
# Convert DataFrame to list of dictionaries with correct keys
data_to_insert = data_from_csv.rename(columns={
    'Title': 'topic_name',
    'Year': 'year',
    'Level': 'level',
    'Introduction Summary': 'introduction_summary',
    'Learning Outcomes': 'learning_outcomes',
    'Link to Summary Page': 'summary_page_link',
    'Link to PDF File': 'pdf_file_link'
}).to_dict(orient='records')

# Insert data into Snowflake
for data in data_to_insert:
    insert_data(data)

print("Data upload completed.")

Data upload completed.


You can now verify the above upload process by opening the web-based interface of Snowflake, by going into the databases and the table will be displayed there with the uploaded data.

We can also run Step 5 and 6 commands on the terminal to verify.